In [1]:
import torch
import copy
import numpy as np
from models.baselines.LSTM import LSTM_Classification
from utils.evaluators import compute_acc, compute_ave_batch_loss
from datasets.process import ecg5000_reading
from sklearn.model_selection import train_test_split
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader


RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

X, Y = ecg5000_reading()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=RANDOM_SEED, stratify=Y)
X_verify, X_test, Y_verify, Y_test = train_test_split(X_test, Y_test, test_size=0.5, random_state=RANDOM_SEED, stratify=Y_test)

class Kaggle_ECG(Dataset):
    def __init__(self, X, Y):
        self.x = X
        self.y = Y
        self.length = len(self.y)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.length
    
ds_train = Kaggle_ECG(X_train, Y_train)
ds_verify = Kaggle_ECG(X_verify, Y_verify)
ds_test = Kaggle_ECG(X_test, Y_test)

/root/autodl-tmp/datasets/process.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train.append(test, ignore_index=True)


In [3]:
_n_epoch = 100
_batch_size = 200
train_data_loader = DataLoader(dataset=ds_train, batch_size=_batch_size, shuffle=True)
verify_data_loader = DataLoader(dataset=ds_verify, batch_size=_batch_size, shuffle=True)
test_data_loader = DataLoader(dataset=ds_test, batch_size=_batch_size, shuffle=True)

model = LSTM_Classification(1, hidden_dim=100, target_size=5).cuda()
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [4]:
import warnings
warnings.filterwarnings ("ignore")
for epoch in range(_n_epoch):
    
    if epoch == 0:
        acc, prec, f1 = compute_acc(model, test_data_loader, labels=[0, 1, 2, 3, 4])
        print('🌟🌟🌟 Zero-shot Result: \tAccuracy: {:.6f} \tPrecision: {:.6f} \tF1-score: {:.6f}'.format(
            acc, prec, f1
        ))
    
    train_loss = 0
    for i, data in enumerate(train_data_loader):
        optimizer.zero_grad()
        data_t, target = data[0].cuda(), data[1].reshape(_batch_size)
        o_ = model(data_t)
        loss = criterion(torch.log(o_), target.cuda())
        
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * _batch_size
        
    epoch_train_loss = train_loss / len(train_data_loader)
    
    epoch_verify_loss = compute_ave_batch_loss(model, verify_data_loader, criterion)
    
    epoch_test_loss = compute_ave_batch_loss(model, test_data_loader, criterion)
    
    acc, prec, f1 = compute_acc(model, test_data_loader, labels=[0, 1, 2, 3, 4])
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}  \tTesting Loss: {:.6f} '
         ' \n\t\tAccuracy: {:.6f} \tPrecision: {:.6f} \tF1-score: {:.6f}'.format(
        epoch, 
        epoch_train_loss,
        epoch_verify_loss,
        epoch_test_loss,
        acc, prec, f1
    ))

🌟🌟🌟 Zero-shot Result: 	Accuracy: 0.006000 	Precision: 0.004569 	F1-score: 0.006320
Epoch: 0 	Training Loss: 297.628411 	Validation Loss: 239.915972  	Testing Loss: 241.055274  
		Accuracy: 0.832000 	Precision: 0.330574 	F1-score: 0.341671
Epoch: 1 	Training Loss: 151.561379 	Validation Loss: 108.549573  	Testing Loss: 104.929440  
		Accuracy: 0.860000 	Precision: 0.346937 	F1-score: 0.350995
Epoch: 2 	Training Loss: 99.762742 	Validation Loss: 91.890728  	Testing Loss: 91.581198  
		Accuracy: 0.900000 	Precision: 0.356678 	F1-score: 0.369412
Epoch: 3 	Training Loss: 89.986581 	Validation Loss: 85.327096  	Testing Loss: 83.066421  
		Accuracy: 0.909000 	Precision: 0.361153 	F1-score: 0.373080
Epoch: 4 	Training Loss: 85.053478 	Validation Loss: 81.425483  	Testing Loss: 79.172539  
		Accuracy: 0.911000 	Precision: 0.362000 	F1-score: 0.373956
Epoch: 5 	Training Loss: 80.900964 	Validation Loss: 76.874881  	Testing Loss: 74.647582  
		Accuracy: 0.916000 	Precision: 0.364445 	F1-score: 0.